This notebook demonstrates some of the functionality of the AEIC configuration system (defined in `AEIC.config`).

### Imports

Import just for setting AEIC_PATH environment variable and for doing some path manipulations later.

In [1]:
import os
from pathlib import Path

The AEIC configuration class is called `Config`, and `config` (lower-case) is the singleton configuration object.

In [2]:
from AEIC.config import Config, config

### AEIC path

We only need to set the AEIC path if we want to use data from places other than the `data` directory included in the project (which is in `src/aeic/data` for reasons of Python package layout). That `data` directory is always included as a fallback on the AEIC path. Here's an example of adding another directory:

In [3]:
os.environ['AEIC_PATH'] = '/big/AEIC/data'

### Configuration initialization

If we try to access configuration values before setting up the configuration, we get an error.

In [4]:
config.path

ValueError: AEIC configuration is not set

Load the default configuration. We can supply a path to an "overlay" TOML file that includes only configuration items we want to override relative to the defaults. We can also specify overrides for configuration items using keyword arguments to `Config.load` (see below).

In [5]:
cfg = Config.load()

Here's what the configuration looks like.

In [6]:
cfg

Config(path=[PosixPath('/big/AEIC/data'), PosixPath('/home/iross/work/mit/code/AEIC/src/AEIC/data')], data_path_overrides=[], performance_model=PosixPath('/home/iross/work/mit/code/AEIC/src/AEIC/data/performance/sample_performance_model.toml'), engine_file=PosixPath('/home/iross/work/mit/code/AEIC/src/AEIC/data/engines/sample_edb.xlsx'), weather=WeatherConfig(use_weather=True, weather_data_dir=PosixPath('/home/iross/work/mit/code/AEIC/src/AEIC/data/weather')), emissions=EmissionsConfig(fuel='conventional_jetA', climb_descent_mode=<ClimbDescentMode.TRAJECTORY: 'trajectory'>, co2_enabled=True, h2o_enabled=True, sox_enabled=True, nox_method=<EINOxMethod.BFFM2: 'bffm2'>, hc_method=<EINOxMethod.BFFM2: 'bffm2'>, co_method=<EINOxMethod.BFFM2: 'bffm2'>, pmvol_method=<PMvolMethod.FUEL_FLOW: 'fuel_flow'>, pmnvol_method=<PMnvolMethod.MEEM: 'meem'>, apu_enabled=True, gse_enabled=True, lifecycle_enabled=True))

### Configuration immutability

The configuration is frozen: we cannot modify it once it's set up. This is intended to avoid problems with simulations running with inconsistent configuration settings. (This does mean that some special tricks are needed for modifying configuration data during tests, but it's worth the small amount of additional effort to have immutable configuration data for non-test use cases.)

In [7]:
cfg.weather.use_weather = False

ValidationError: 1 validation error for WeatherConfig
use_weather
  Instance is frozen [type=frozen_instance, input_value=False, input_type=bool]
    For further information visit https://errors.pydantic.dev/2.12/v/frozen_instance

### Configuration access

As well as accessing configuration items from the `Config` value returned by `Config.load`, we can also access them via the global `config` value. That's the normal way to get access to AEIC configuration items.

In [8]:
cfg.emissions.fuel_file

PosixPath('/home/iross/work/mit/code/AEIC/src/AEIC/data/fuels/conventional_jetA.toml')

In [9]:
config.emissions.fuel_file

PosixPath('/home/iross/work/mit/code/AEIC/src/AEIC/data/fuels/conventional_jetA.toml')

### Other configuration operations

You should not need to do this much, but it's possible to *reset* the configuration so that you can load a different one. This is important for testing and interactive use of the configuration system, but you should not be doing it in "normal" AEIC code!

In [10]:
Config.reset()

We can override configuration items using keyword arguments to `Config.load`: here we set the `lifecycle_enabled` emissions option to `False` (it defaults to `True`) and set the LTO input model to "performance_model" (it defaults to "edb", and you can use a string or the `LTOInputMode` enum value). All other values are taken from the default configuration that's included in the AEIC package.

In [11]:
Config.load(
    emissions=dict(lifecycle_enabled=False),
    data_path_overrides=[Path('/big/my-aeic-data')],
)

Config(path=[PosixPath('/big/AEIC/data'), PosixPath('/home/iross/work/mit/code/AEIC/src/AEIC/data')], data_path_overrides=[PosixPath('/big/my-aeic-data')], performance_model=PosixPath('/home/iross/work/mit/code/AEIC/src/AEIC/data/performance/sample_performance_model.toml'), engine_file=PosixPath('/home/iross/work/mit/code/AEIC/src/AEIC/data/engines/sample_edb.xlsx'), weather=WeatherConfig(use_weather=True, weather_data_dir=PosixPath('/home/iross/work/mit/code/AEIC/src/AEIC/data/weather')), emissions=EmissionsConfig(fuel='conventional_jetA', climb_descent_mode=<ClimbDescentMode.TRAJECTORY: 'trajectory'>, co2_enabled=True, h2o_enabled=True, sox_enabled=True, nox_method=<EINOxMethod.BFFM2: 'bffm2'>, hc_method=<EINOxMethod.BFFM2: 'bffm2'>, co_method=<EINOxMethod.BFFM2: 'bffm2'>, pmvol_method=<PMvolMethod.FUEL_FLOW: 'fuel_flow'>, pmnvol_method=<PMnvolMethod.MEEM: 'meem'>, apu_enabled=True, gse_enabled=True, lifecycle_enabled=False))

In [12]:
config.emissions.lifecycle_enabled

False

In [13]:
config.data_path_overrides

[PosixPath('/big/my-aeic-data')]

You can get a reference to the current configuration with `Config.get()`. This can be useful if you want to see the full configuration object, instead of accessing through the `config` proxy.

In [14]:
Config.get()

Config(path=[PosixPath('/big/AEIC/data'), PosixPath('/home/iross/work/mit/code/AEIC/src/AEIC/data')], data_path_overrides=[PosixPath('/big/my-aeic-data')], performance_model=PosixPath('/home/iross/work/mit/code/AEIC/src/AEIC/data/performance/sample_performance_model.toml'), engine_file=PosixPath('/home/iross/work/mit/code/AEIC/src/AEIC/data/engines/sample_edb.xlsx'), weather=WeatherConfig(use_weather=True, weather_data_dir=PosixPath('/home/iross/work/mit/code/AEIC/src/AEIC/data/weather')), emissions=EmissionsConfig(fuel='conventional_jetA', climb_descent_mode=<ClimbDescentMode.TRAJECTORY: 'trajectory'>, co2_enabled=True, h2o_enabled=True, sox_enabled=True, nox_method=<EINOxMethod.BFFM2: 'bffm2'>, hc_method=<EINOxMethod.BFFM2: 'bffm2'>, co_method=<EINOxMethod.BFFM2: 'bffm2'>, pmvol_method=<PMvolMethod.FUEL_FLOW: 'fuel_flow'>, pmnvol_method=<PMnvolMethod.MEEM: 'meem'>, apu_enabled=True, gse_enabled=True, lifecycle_enabled=False))